In [2]:
import pandas as pd
import re

In [3]:
!ls

DATA                          README.md
DATA-20230703T120413Z-001.zip filebaru.txt
Filebaru2.txt                 train_preprocess.tsv.txt
MembuatModel.ipynb


In [4]:
df_tweet_train = pd.read_csv("train_preprocess.tsv.txt",header=None,names=['tweet','sentiment'],delimiter='\t')
df_tweet_train

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
...,...,...
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive
10997,hormati partai-partai yang telah berkoalisi,neutral
10998,"pagi pagi di tol pasteur sudah macet parah , b...",negative


In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nlp_id.lemmatizer import Lemmatizer 
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

alay_dict=pd.read_csv("DATA/new_kamusalay.csv",encoding='latin1',header=None)
alay_dict=alay_dict.rename(columns={0:'Original',1:'Baku'})

kasar_dict=pd.read_csv("DATA/abusive.csv",encoding='latin1')
kasar_dict['Kata_Sensor']="a1b2c3d4e5f6" #Inisiasi kata ganti untuk kata-kata yang kasar dengan kata "disensor"

print(kasar_dict)

alay_dict_map = dict(zip(alay_dict['Original'], alay_dict['Baku']))
kasar_dict_map = dict(zip(kasar_dict['ABUSIVE'],kasar_dict['Kata_Sensor']))

def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])


def sensor_kata_kasar(text):
    return ' '.join([kasar_dict_map[word] if word in kasar_dict_map else word for word in text.split(' ')])


def preprocess(TextYangInginDiPreProcess):
    #Tahap Pertama Adalah Membuat semua huruf menjadi huruf kecil atau lower
    text = TextYangInginDiPreProcess.lower()

    #Tahap Kedua adalah menghilangkan non alpha numeric character pada text
    text = re.sub('[^0-9a-zA-Z]+',' ',text)

    #Tahap Ketiga adalah menghilangkan char tidak penting
    text=re.sub('\n',' ',text) #Menghilangkan new line pada data
    text=re.sub('rt',' ',text) #Menghilangkan kata-kata retweet 
    text=re.sub('user',' ',text) #Menghilangkan kata-kata user
    text=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))',' ',text) #Menghilangkan  URL
    text=re.sub(' +',' ',text) #Menghilangkan ekstra spasi

    #Tahap keempat adalah membuat map terhadap kata-kata "alay" dan mengubah nya menjadi kata yang baku
    text=normalize_alay(text)

    #Tahap kelima adalah mensensor kata kasar dengan kata "disensor"
    text=sensor_kata_kasar(text)
    return text

def menghilangkan_kata_umum_dengan_stopword(text):
    # Mengambil daftar stopwords bahasa Indonesia dari NLTK
    stop_words = set(stopwords.words('indonesian'))
    
    # Memisahkan teks menjadi kata-kata
    words = text.split()
    
    # Menghapus stopwords dari teks
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Menggabungkan kata-kata yang tersisa menjadi teks tanpa stopwords
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

def mengurangi_variasi_kata_dengan_lematisasi(teks):
    lemmatizer = Lemmatizer() 
    teks = lemmatizer.lemmatize(teks) 
    return teks

def preprocess_semua_kata(teks):
    teks = preprocess(teks)
    teks = menghilangkan_kata_umum_dengan_stopword(teks)
    teks = mengurangi_variasi_kata_dengan_lematisasi(teks)
    return teks

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


         ABUSIVE   Kata_Sensor
0           alay  a1b2c3d4e5f6
1          ampas  a1b2c3d4e5f6
2           buta  a1b2c3d4e5f6
3        keparat  a1b2c3d4e5f6
4         anjing  a1b2c3d4e5f6
..           ...           ...
120        rezim  a1b2c3d4e5f6
121        sange  a1b2c3d4e5f6
122       serbet  a1b2c3d4e5f6
123        sipit  a1b2c3d4e5f6
124  transgender  a1b2c3d4e5f6

[125 rows x 2 columns]


In [6]:
kata_kata = "Dasar Anak Anjing"
print(preprocess_semua_kata(kata_kata))


dasar anak a1b2c3d4e5f6


In [8]:
df_tweet_train[df_tweet_train['sentiment']=='positive']

,tweet,sentiment
0,warung ini dimiliki oleh pengusaha pabrik tahu...,positive
2,lokasi strategis di jalan sumatera bandung . t...,positive
3,betapa bahagia nya diri ini saat unboxing pake...,positive
5,"makanan beragam , harga makanan di food stall ...",positive
7,"tempat unik , bagus buat foto , makanan enak ,...",positive
...,...,...
10992,kalau memang jala nya sudah begitu terima saja...,positive
10994,tidak bosan,positive
10995,tidak kecewa,positive
10996,enak rasa masakan nya apalagi kepiting yang me...,positive


In [9]:
df_tweet_train[df_tweet_train['sentiment']=='neutral']

,tweet,sentiment
1,mohon ulama lurus dan k212 mmbri hujjah partai...,neutral
16,setiap bumn dibentuk dengan uu bukan dibentuk ...,neutral
28,pdip sebut ridwan kamil menang karena berbaju ...,neutral
32,08:30 : kedatangan presiden ri dan rombongan d...,neutral
37,risma menjatuhkan pilihan nya pada gus ipul - ...,neutral
...,...,...
10925,pkb benarkan charly setia band daftar jadi caw...,neutral
10967,pernah percaya kalau body lotion citra bikin b...,neutral
10969,"bu susi marah , menteri luhut perbolehkan kapa...",neutral
10993,f - demokrat dorong upaya kemandirian energi n...,neutral


In [10]:
df_tweet_train[df_tweet_train['sentiment']=='negative']

,tweet,sentiment
4,duh . jadi mahasiswa jangan sombong dong . kas...,negative
6,pakai kartu kredit bca tidak untung malah rugi...,negative
10,simcard indosat inaktiv gara-gara lupa isi pul...,negative
11,sifat iri sering muncul pada orang orang yang ...,negative
13,"pengalaman bersama indosat hari ini , semoga t...",negative
...,...,...
10982,tolong dibaca pesan saya ! di dalam kotak hand...,negative
10985,"sumpah gue kecewa banget dengan sms , cuma tea...",negative
10989,tadinya pengin mengantar teman dari kalau ke r...,negative
10991,dapat rom awal abad,negative
